------------------------------------------------------------------------

Copyright 2023 Benjamin Alexander Albert \[Karchin Lab\]

All Rights Reserved

BigMHC Academic License

makeseqs.ipynb

------------------------------------------------------------------------


#### Generate MHC pseudosequences from FASTA files

Create a dir, which we will call `data`
  * Create a dir within `data` called `seq`
    * Store all of the FASTA files and SAM alignment files the `seq` dir
  * The final `pseudoseqs.csv`file will be placed in the `data` dir


Three protein FASTA files are required as input:

1. HLA FASTA file retrieved from: https://github.com/ANHIG/IMGTHLA
   
   Robinson J, Barker DJ, Georgiou X, Cooper MA, Flicek P, Marsh SG.
   IPD-IMGT/HLA Database.
   Nucleic acids research. 2020 Jan 8;48(D1):D948-55.
   doi:10.1093/nar/gkz950

   Robinson J, Malik A, Parham P, Bodmer JG, Marsh SG.
   IMGT/HLA database: a sequence database for the human major histocompatibility complex.
   Tissue antigens. 2000 Mar;55(3):280-7.
   doi:10.1034/j.1399-0039.2000.550314.x

   
2. non-HLA MHC FASTA file retrieved from: https://github.com/ANHIG/IPDMHC

   Robinson J, Halliwell JA, Hayhurst JD, Flicek P, Parham P, Marsh SG.
   The IPD and IMGT/HLA database: allele variant databases.
   Nucleic acids research. 2015 Jan 28;43(D1):D423-31.
   doi:10.1093/nar/gku1161


3. H2 data must be curated from UniProt:


   | Allele | UniProt_Accession_ID | FASTA_URL                                    |
   |--------|----------------------|----------------------------------------------|
   | H2-Db  | P01899               | https://www.uniprot.org/uniprot/P01899.fasta |
   | H2-Dd  | P01900               | https://www.uniprot.org/uniprot/P01900.fasta |
   | H2-Dp  | P14427               | https://www.uniprot.org/uniprot/P14427.fasta |
   | H2-Dk  | P14426               | https://www.uniprot.org/uniprot/P14426.fasta |
   | H2-Dq  | Q31145               | https://www.uniprot.org/uniprot/Q31145.fasta |
   | H2-Kb  | P01901               | https://www.uniprot.org/uniprot/P01901.fasta |
   | H2-Kd  | P01902               | https://www.uniprot.org/uniprot/P01902.fasta |
   | H2-Kk  | P04223               | https://www.uniprot.org/uniprot/P04223.fasta |
   | H2-Kq  | P14428               | https://www.uniprot.org/uniprot/P14428.fasta |
   | H2-Ld  | P01897               | https://www.uniprot.org/uniprot/P01897.fasta |
   | H2-Lq  | Q31151               | https://www.uniprot.org/uniprot/Q31151.fasta |

In [ ]:
import os

datadir = os.path.join(os.pardir, "data")
seqdir = os.path.join(datadir, "seq")

#### Compile all MHC sequences into a single FASTA file

In [ ]:
"""
Generate a mapping from MHC allele to variable-length MHC protein sequence.
The mappings are saved to compiled.fasta in the seq dir
"""

import re


def getMappings(fp, prefix=""):
    """
    Read FASTA file at location fp to extract allele-protein mappings.
    FASTA description lines should be white-space separated such that
    the second token is the MHC allele name. If only one token exists,
    then the sole token is threated as the MHC allele name.
    MHC proteins can span multiple lines.
    """
    maps = dict()
    with open(fp, 'r') as f:
        allele = None
        for line in [x.strip() for x in f.readlines()]:
            if line.startswith('>'):
                if allele is not None:
                    maps[allele] = seq
                toks = line.split()
                name = toks[1] if len(toks) > 1 else toks[0][1:]
                allele = prefix + name
                seq = str()
            else:
                seq += line
    return maps


def extractSupertypes(rawmaps):
    """
    Only supertypes are mapped (e.g. HLA-A\*02:01 instead of HLA-A\*02:01:01:02N).
    If multiple sequences under a single supertype are received, then the
    most frequent MHC protein sequence is selected. Ties are broken by choosing
    the longer sequence to preserve as much information as possible.
    Alleles with "-N" in the name are excluded such as Eqca-N*001:01 and BoLA-NC1*002:01.
    """
    supmaps = dict()
    supreg  = r"[^:]+:[^:]+"
    for mhc,seq in rawmaps.items():
        match = re.match(supreg, mhc)
        if match is None:
            continue
        if "-N" in mhc:
            continue
        supmhc = match.group()
        if supmhc not in supmaps:
            supmaps[supmhc] = {seq:0}
        else:
            if seq in supmaps[supmhc]:
                supmaps[supmhc][seq] += 1
            else:
                supmaps[supmhc][seq] = 0
    maxmaps = dict()
    for mhc,seqdict in supmaps.items():
        maxcnt = 0
        maxseq = str()
        for seq,cnt in seqdict.items():
            if cnt > maxcnt or (cnt==maxcnt and len(seq) > len(maxseq)):
                maxcnt = cnt
                maxseq = seq
        supmaps[mhc] = maxseq
        maxmaps[mhc] = maxseq
    return maxmaps


def filterClass(rawmaps):
    """
    Extracts MHC-I molecules and stores them in dicts.
    Valid alleles either start with H2 or end in a digit.
    All MHC-II alleles contain "-D", and the rest are MHC-I.
    """
    mhcmaps = dict()
    for mhc,seq in rawmaps.items():
        if "-MIC" in mhc:
            continue
        if "-TAP" in mhc:
            continue
        if "-D" in mhc:
            continue
        if (not mhc.startswith("H2")) and mhc[-1].isalpha():
            continue
        mhcmaps[mhc] = seq
    return mhcmaps


def writemaps(fp, maps):
    lines = [">{}\n{}".format(k,v) for k,v in maps.items()]
    lines.sort()
    with open(fp, 'w') as f:
        f.writelines('\n'.join(lines))


maps = dict()
maps.update(getMappings(os.path.join(seqdir, "hla.fasta"), prefix="HLA-"))
maps.update(getMappings(os.path.join(seqdir, "mhc.fasta")))
maps = extractSupertypes(maps)
maps = filterClass(maps)
maps.update(getMappings(os.path.join(seqdir, "h2.fasta")))
print("writing compiled fasta with {} lines...".format(len(maps)), end="")
writemaps(os.path.join(seqdir, "compiled.fasta"),  maps)
print("done")

#### Align the compiled FASTA file and extract pseudosequences

Each pseudosequence is designed to be a shortened representation of the amino acid residues in an MHC allele, which requires identifying the most important residue positions. The BigMHC approach uses a multiple sequence alignment (MSA) and calculates the relative entropy of each residue position when aligned to a set of MHC alleles. We consider the positions with the largest relative entropy to have the most information, from which we select the 30 most entropic positions. These calculations were performed using SAM suite 3.5.

First, run SAM's `buildmodel` on a fasta file of unaligned sequences, such as the `compiled.fasta` file in the seq dir. You can use any set of unaligned sequences that you believe are relevant. `buildmodel` will generate a hidden Markov model of your sequences, after which you can construct an MSA using `align2model`, make a pretty alignment using `prettyalign`, and extract relative entropies for all positions using `makelogo`. This is our current method of computing pseudosequences, and although imprecision may arise, the resulting pseudosequences are comparable to the original implementation.

Specifically, perform the following:
1. Download the SAM source code: `wget https://karchinlab.org/datasets/sam-src.tar.gz`
2. Extract the contents: `tar -xzvf sam-src.tar.gz`
3. Within the SAM directory, configure the build: `./configure`
4. Compile with GNU Make: `make`
5. The executables are deposited in the SAM `src` directory, in which you can run the following commands:
6. `./buildmodel mhc -train <PATH_TO_FASTA>`
7. `./align2model mhc -modelfile mhc.mod -db <PATH_TO_FASTA>`
8. `./prettyalign mhc.a2m -m0 > mhc.pretty`
9. `./makelogo mhc -i mhc.mod &> mhc.bits`

Note: the authors of SAM no longer develop or maintain the package; the contact information included in the package is obsolete, and we cannot provide any support for SAM installation or usage ourselves.

In [ ]:
"""
Extract pseudo-sequences from SAM prettyalign output.

Deletions are first replaced with dummy residue 'X'.
Then, pseudosequences are extracted by taking the top n
positions, sorted by entropy, where n is parameterized.

The extracted components are onehot encoded before
the resulting pseudosequences are saved to a csv file.
"""

import pandas as pd


def extractPseudoseqs(inp):
    aligns = dict()
    with open(inp, 'r') as f:
        for line in f.readlines():
            if (line[0].isalpha() or line[0].isdigit()) and ' ' in line:
                mhc = line[:line.find(' ')].strip()
                seq = list(line[line.find(' '):].strip().replace('-','X'))
                if mhc not in aligns:
                    aligns[mhc] = seq
                else:
                    aligns[mhc] += seq
    return pd.DataFrame.from_dict(aligns, orient="index")


def encodePseudoseqs(seqs, bits, cmps):
    with open(bits, 'r') as f:
        bit = [float(x.strip()) for x in f.readlines() if x[0].isdigit()]
    rmi = [seqs.iloc[:,x].unique().shape[0]==1 for x in range(seqs.shape[1])]
    idx = [x for x in range(len(rmi)) if not rmi[x]]
    bit = [bit[x] for x in range(len(bit)) if not rmi[x]]
    seqs.drop(seqs.columns[[x for x in range(seqs.shape[1]) if rmi[x]]], axis=1, inplace=True)
    srt = [x[0] for x in sorted(enumerate(bit), key=lambda x: x[1])]
    for x in srt[:cmps]:
        print("{}\t{}".format(idx[x], ','.join(seqs.iloc[:,x].value_counts(ascending=False).index.values)))
    seqs.drop(seqs.columns[srt[cmps:]], axis=1, inplace=True)
    seqs = pd.get_dummies(seqs, drop_first=False, dtype=int)
    print("dimensionality: {}".format(seqs.shape[1]))
    return seqs

seqs = extractPseudoseqs(os.path.join(seqdir, "mhc.pretty"))
encs = encodePseudoseqs(seqs, os.path.join(seqdir, "mhc.bits"), 30)
encs.to_csv(os.path.join(datadir, "pseudoseqs.csv"), index_label="mhc")